In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [4]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler

In [5]:
import matplotlib.pyplot as plt 
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')

In [6]:
Cough_dataframe = pd.read_csv("./Cough.csv")
Wheeze_dataframe = pd.read_csv("./Wheeze.csv")
Aaa_dataframe = pd.read_csv("./Aaa.csv")
Yee_dataframe = pd.read_csv("./Yee.csv")
Inhale_dataframe = pd.read_csv("./Inhale.csv")
Exhale_dataframe = pd.read_csv("./Exhale.csv")
Ooo_dataframe = pd.read_csv("./Ooo.csv")
Sss_dataframe = pd.read_csv("./Sss.csv")
Eee_dataframe = pd.read_csv("./Eee.csv")
Uuu_dataframe = pd.read_csv("./Uuu.csv")
Zzz_dataframe = pd.read_csv("./Zzz.csv")

In [7]:
sets_loc = './SETS/'

In [8]:
def fold(sound_dataframe, sets_loc):
    big = []
    for j in os.listdir(sets_loc):      

        file = open(sets_loc+j, 'r') 
        Lines = file.readlines() 

        count = 0
        # Strips the newline character 
        # for line in Lines: 
        #     print("Line{}: {}".format(count, line.strip())) 

        set_dataframe = pd.DataFrame()
        for i, line in enumerate(Lines):
            A = sound_dataframe[(sound_dataframe['79'] == (Lines[i].strip()+".json"))]
            set_dataframe = pd.DataFrame.append(set_dataframe,A)
        
        big.append(set_dataframe)
    fold_1 = shuffle(pd.concat(big[:4]))
    test_1 = shuffle(big[4])
    fold_2 = shuffle(pd.concat(big[1:5]))
    test_2 = shuffle(big[0])
    fold_3 = shuffle(pd.concat([big[0],big[2],big[3],big[4]]))
    test_3 = shuffle(big[1])
    fold_4 = shuffle(pd.concat([big[0],big[1],big[3],big[4]]))
    test_4 = shuffle(big[2])
    fold_5 = shuffle(pd.concat([big[0],big[2],big[1],big[4]]))
    test_5 = shuffle(big[3])
    
    return fold_1,test_1,fold_2,test_2,fold_3,test_3,fold_4,test_4,fold_5,test_5

In [9]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Cough_dataframe,sets_loc)

In [10]:
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]

In [11]:
def SVM(fold_list):
    list_train = []
    list_test = []
    for i in range(0,len(fold_list),2):
        fold = fold_list[i]
        test = fold_list[i+1]
        X_train = fold.iloc[:,:78]
        y_train = fold.iloc[:,78]
        X_test = test.iloc[:,:78]
        y_test = test.iloc[:,78]


        scaler = StandardScaler()

        X_train = scaler.fit_transform(X_train)

        X_test = scaler.transform(X_test)

        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)



        ### SVM

        svm = SVC()

        svm.fit(X_train,y_train)

        list_test.append(accuracy_score(y_test,svm.predict(X_test)))

        list_train.append(accuracy_score(y_train,svm.predict(X_train)))
        
    return list_test, list_train

In [12]:
cough_test, cough_train = SVM(fold_list)

In [13]:
from sklearn.model_selection import StratifiedShuffleSplit

In [14]:
def hyperparameter_tuning(fold,test):
    X_train = fold.iloc[:,:78]
    y_train = fold.iloc[:,78]    
    C_range = np.logspace(-1, 10, 12)
    gamma_range = np.logspace(-1, 20, 22)
    param_grid = dict(gamma=gamma_range, C=C_range)
    cv = StratifiedShuffleSplit(n_splits=5)
    grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv, verbose=3, n_jobs=-1)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    grid.fit(X_train, y_train)
    print(grid.best_params_)
    X_test = test.iloc[:,:78]
    X_test = scaler.transform(X_test)
    y_pred = grid.predict(X_test)
    a = accuracy_score(test.iloc[:,78],y_pred)
    return a


In [53]:
tune_acc_cough = []
for i in range(0,len(fold_list),2):
    tune_acc_cough.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 278 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 918 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   10.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.9s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.3s finished


In [57]:
np.mean(cough_test)

0.6676646182353787

In [20]:
cough_test

[0.6612903225806451,
 0.7209302325581395,
 0.589622641509434,
 0.668918918918919,
 0.697560975609756]

In [21]:
np.std(cough_test)

0.044415701953316386

In [58]:
(tune_acc_cough)

[0.6896551724137931,
 0.6046511627906976,
 0.49523809523809526,
 0.5172413793103449,
 0.47474747474747475]

In [59]:
np.mean(tune_acc_cough)

0.5563066569000812

In [60]:
a = [1,2,3,4]

In [61]:
np.std(tune_acc_cough)

0.08003297216716057

### Wheeze

In [62]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Wheeze_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
wheeze_test, wheeze_train = SVM(fold_list)


In [63]:
wheeze_test, wheeze_train

([0.8881578947368421,
  0.7651515151515151,
  0.7007299270072993,
  0.6608695652173913,
  0.6],
 [0.9744597249508841,
  0.9735349716446124,
  0.982824427480916,
  0.9688644688644689,
  0.9794776119402985])

In [64]:
np.mean(wheeze_test)

0.7229817804226095

In [65]:
np.std(wheeze_test)

0.09852819478817772

In [66]:
tune_acc_wheeze = []
for i in range(0,len(fold_list),2):
    tune_acc_wheeze.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 1040 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   16.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 1040 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   16.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 528 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   17.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 1040 tasks      | elapsed:   13.0s


{'C': 1.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   16.5s finished


In [67]:
tune_acc_wheeze

[0.42105263157894735,
 0.6363636363636364,
 0.5693430656934306,
 0.5652173913043478,
 0.528]

In [68]:
np.mean(tune_acc_wheeze)

0.5439953449880723

In [69]:
np.std(tune_acc_wheeze)

0.07070437792404904

### Aaa 

In [23]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Aaa_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Aaa_test, Aaa_train = SVM(fold_list)


In [27]:
(Aaa_test)

([0.5257731958762887,
  0.6024096385542169,
  0.6320754716981132,
  0.42528735632183906,
  0.7128712871287128],)

In [28]:
np.std(Aaa_test)

0.09771885466539448

In [72]:
tune_acc_Aaa = []
for i in range(0,len(fold_list),2):
    tune_acc_Aaa.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 1.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    9.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    9.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 1.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.8s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.6s finished


In [73]:
np.mean(tune_acc_Aaa)

0.5347037183808909

In [74]:
tune_acc_Aaa

[0.5463917525773195,
 0.5662650602409639,
 0.5188679245283019,
 0.5172413793103449,
 0.5247524752475248]

In [75]:
np.std(tune_acc_Aaa)

0.01890746591186171

### Eee

In [76]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Eee_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Eee_test, Eee_train = SVM(fold_list)


In [77]:
Eee_test, Eee_train

([0.7586206896551724,
  0.6046511627906976,
  0.6571428571428571,
  0.6551724137931034,
  0.6868686868686869],
 [0.9628647214854111,
  0.9629629629629629,
  0.9721448467966574,
  0.9681697612732095,
  0.9616438356164384])

In [78]:
np.mean(Eee_test)

0.6724911620501034

In [79]:
tune_acc_Eee = []
for i in range(0,len(fold_list),2):
    tune_acc_Eee.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 400 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1040 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s


{'C': 1.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 1.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.7s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.3s finished


In [80]:
tune_acc_Eee

[0.6896551724137931,
 0.6046511627906976,
 0.49523809523809526,
 0.5172413793103449,
 0.47474747474747475]

In [81]:
np.mean(tune_acc_Eee)

0.5563066569000812

In [82]:
np.std(Eee_test)

0.05052283394587567

### Yee

In [83]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Yee_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Yee_test, Yee_train = SVM(fold_list)

In [84]:
Yee_test, Yee_train

([0.8658536585365854,
  0.6091954022988506,
  0.7019230769230769,
  0.6511627906976745,
  0.6263736263736264],
 [0.9456521739130435,
  0.9724517906336089,
  0.9797687861271677,
  0.9697802197802198,
  0.9693593314763231])

In [85]:
np.mean(Yee_test)

0.6909017109659629

In [86]:
tune_acc_Yee = []
for i in range(0,len(fold_list),2):
    tune_acc_Yee.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 1.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.7s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.0s finished


In [87]:
tune_acc_Yee

[0.5609756097560976,
 0.4942528735632184,
 0.5192307692307693,
 0.5465116279069767,
 0.4175824175824176]

In [88]:
np.mean(tune_acc_Yee)

0.507710659607896

In [89]:
np.mean(Yee_test)


0.6909017109659629

In [90]:
np.std(Yee_test)

0.09289631944007794

### Inhale

In [29]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Inhale_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Inhale_test, Inhale_train = SVM(fold_list)


In [ ]:
tune_acc_Inhale = []
for i in range(0,len(fold_list),2):
    tune_acc_Inhale.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

In [30]:
np.mean(tune_acc_Inhale)

NameError: name 'tune_acc_Inhale' is not defined

In [31]:
np.mean(Inhale_test)

0.7650516491527778

In [32]:
Inhale_test

[0.8121546961325967,
 0.7848101265822784,
 0.7473118279569892,
 0.6809815950920245,
 0.8]

In [94]:
tune_acc_Inhale

[0.6132596685082873,
 0.5379746835443038,
 0.6182795698924731,
 0.754601226993865,
 0.6114285714285714]

In [95]:
np.std(tune_acc_Inhale)

0.07030623439287106

### Exhale

In [33]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Exhale_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Exhale_test, Exhale_train = SVM(fold_list)


In [ ]:
tune_acc_Exhale = []
for i in range(0,len(fold_list),2):
    tune_acc_Exhale.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

In [34]:
np.mean(Exhale_test)

0.7018601215206426

In [35]:
Exhale_test

[0.7734806629834254,
 0.7388535031847133,
 0.6935483870967742,
 0.6748466257668712,
 0.6285714285714286]

In [36]:
np.std(Exhale_test)

0.05033499506812715

In [98]:
tune_acc_Exhale

[0.7458563535911602,
 0.7261146496815286,
 0.5913978494623656,
 0.6932515337423313,
 0.5771428571428572]

In [99]:
np.mean(tune_acc_Exhale)

0.6667526487240485

In [100]:
np.std(tune_acc_Exhale)

0.0695581694807795

### Ooo

In [101]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Ooo_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Ooo_test, Ooo_train = SVM(fold_list)
tune_acc_Ooo = []
for i in range(0,len(fold_list),2):
    tune_acc_Ooo.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   10.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   10.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    4.2s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.9s finished


In [102]:
np.mean(Ooo_test)

0.6541669089288137

In [103]:
Ooo_test

[0.6419753086419753,
 0.6904761904761905,
 0.6730769230769231,
 0.5918367346938775,
 0.673469387755102]

In [104]:
np.std(Ooo_test)

0.034877568688540264

In [105]:
np.mean(tune_acc_Ooo)

0.5452579607341512

### Sss

In [37]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Sss_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Sss_test, Sss_train = SVM(fold_list)
tune_acc_Sss = []
for i in range(0,len(fold_list),2):
    tune_acc_Sss.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   33.2s finished


{'C': 1.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    7.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:   13.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.3s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    7.2s finished


In [38]:
np.mean(Sss_test)

0.7261143116266368

In [40]:
Sss_test

[0.717948717948718,
 0.7662337662337663,
 0.73,
 0.631578947368421,
 0.7848101265822784]

In [41]:
tune_acc_Sss

[0.5512820512820513,
 0.7012987012987013,
 0.59,
 0.631578947368421,
 0.4810126582278481]

In [42]:
np.mean(tune_acc_Sss)

0.5910344716354043

In [43]:
np.std(tune_acc_Sss)

0.07414604417864344

In [44]:
np.std(Sss_test)

0.05305370848670571

### Uuu

In [111]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Uuu_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Uuu_test, Uuu_train = SVM(fold_list)
tune_acc_Uuu = []
for i in range(0,len(fold_list),2):
    tune_acc_Uuu.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done 592 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    8.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    4.6s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    7.7s finished


In [112]:
Uuu_test

[0.5882352941176471,
 0.49411764705882355,
 0.5161290322580645,
 0.620253164556962,
 0.6534653465346535]

In [113]:
np.mean(Uuu_test)

0.5744400969052301

In [114]:
np.std(Uuu_test)

0.06063993885616763

In [115]:
np.mean(tune_acc_Uuu)

0.5267919211396076

### Zzz

In [15]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5 = fold(Zzz_dataframe,sets_loc)
fold_list = [f1,t1,f2,t2,f3,t3,f4,t4,f5,t5]
Zzz_test, Zzz_train = SVM(fold_list)
tune_acc_Zzz = []
for i in range(0,len(fold_list),2):
    tune_acc_Zzz.append(hyperparameter_tuning(fold_list[i],fold_list[i+1]))

Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 383 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    7.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    6.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 10.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


...

[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    5.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s


{'C': 1.0, 'gamma': 0.1}
Fitting 5 folds for each of 264 candidates, totalling 1320 fits


[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:    4.5s


{'C': 10.0, 'gamma': 0.1}


[Parallel(n_jobs=-1)]: Done 1320 out of 1320 | elapsed:    6.0s finished


In [16]:
np.mean(Zzz_test)

0.5425824832276446

In [17]:
tune_acc_Zzz

[0.6172839506172839,
 0.6,
 0.4731182795698925,
 0.5694444444444444,
 0.5230769230769231]

In [18]:
np.mean(tune_acc_Zzz)

0.5565847195417087

In [19]:
np.std(tune_acc_Zzz)

0.0525815077389504